In [ ]:
from google.colab import files
files.upload()

In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c logistics-shopee-code-league

In [ ]:
!unzip delivery_orders_march.csv.zip

In [11]:
import pandas as pd
import datetime
import math

location = ["manila","luzon","visayas","mindanao"]
holiday = [1585065600,1585497600,1585584000]
onSec = 24*60*60
firstSunday = 1578153600
sla = [(3,5,7,7),
       (5,5,7,7),
       (7,7,7,7),
       (7,7,7,7)]

df = pd.read_csv("/content/delivery_orders_march.csv")

def getLocation(str):
  strSplited = str.split(" ")
  return strSplited[len(strSplited)-1].lower()

def fixDate(date):
  return math.floor(date/onSec)*onSec

def countSun(date):

  count = math.floor((date - firstSunday) / (7 * onSec))

  if date - firstSunday % 7 * onSec == 0:
    count = count + 1

  return count

def timeDiffer(start,end):
  start = fixDate(start)
  end = fixDate(end) + onSec
  dateDiff = (end - start) / onSec - 1
  for date in holiday:
    if start <= date and date <= end:
      dateDiff = dateDiff - 1
  dateDiff = dateDiff - (countSun(end - 1) - countSun(start))
  return dateDiff

orderid = []
is_late = []

for row in df.itertuples(index=False):
    start = getLocation(row.selleraddress)
    end = getLocation(row.buyeraddress);
    lim = sla[location.index(start)][location.index(end)]
    orderid.append(row.orderid)
    late = False

  ##  print(str(fixDate(row.pick)) + " " + str(fixDate(row._2)) + " " + timeDeffer(row.pick,row._2) + " " + str(countSun(row.pick)) + " " + str(countSun(row._2)) )

    if math.isnan(row._3):
      if timeDiffer(row.pick,row._2) > lim:
        is_late.append(1)
      else:
        is_late.append(0)
    else:
      if timeDiffer(row._2,row._3) > 3 :
        is_late.append(1)
      else:
        if timeDiffer(row._2,row._3 > lim):
          is_late.append(1)
        else:
          is_late.append(0)

sub = pd.DataFrame({"orderid":orderid,"is_late":is_late})
sub.to_csv(r'submit4.csv', index = False, header=True)

In [12]:
!kaggle competitions submit -c logistics-shopee-code-league -f submit4.csv -m "Null"

100% 41.2M/41.2M [00:07<00:00, 5.44MB/s]
Successfully submitted to [Students] Shopee Code League - Logistics

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from datetime import datetime
import numpy as np
import pandas as pd
from datetime import date
from datetime import timedelta

"""
2020-03-08 (Sunday);
2020-03-25 (Wednesday);
2020-03-30 (Monday);
2020-03-31 (Tuesday)

use pd.isnull(df.something) to check if null
"""

HOLIDAYS = [date(2020, 3, 25), date(2020, 3, 30), date(2020, 3, 30)]
print(HOLIDAYS)

PLACES = ["Metro Manila", "Luzon", "Visayas", "Mindanao"]

SLA_dict = {'Metro Manila Metro Manila': 3 ,'Metro Manila Luzon': 5, 'Metro Manila Visayas': 7,'Metro Manila Mindanao': 7, 
             'Luzon Luzon':5, 'Luzon Visayas': 7, 'Luzon Mindanao':7,
            'Visayas Luzon':7, 'Visayas Visayas':7, 'Visayas Mindanao':7,
            'Mindanao Luzon':7, 'Metro Manila Visayas': 7,'Mindanao Mindanao':7}

df = pd.read_csv("../input/logistics-shopee-code-league/delivery_orders_march.csv")
df['pick'] = pd.to_datetime(df['pick'],unit='s')
df['1st_deliver_attempt'] = pd.to_datetime(df['1st_deliver_attempt'],unit='s')
df['2nd_deliver_attempt'] = pd.to_datetime(df['2nd_deliver_attempt'],unit='s')
df.head()
order_ids = []
is_late = []
for index, row in df.iterrows():
    order_ids.append(row["orderid"])
    
    #get SLA
    for address in PLACES:
        if address in row['buyeraddress']:
            buyeradd = address
        if address in row['selleraddress']:
            selleradd = address
            
    #check for SLA
    add_comb = buyeradd + ' ' + selleradd
    for key in SLA_dict:
        if sorted(add_comb) == sorted(key):
            curr_SLA = SLA_dict[key]
#             print(curr_SLA)
       
    pickup = row["pick"]
    first = row["1st_deliver_attempt"]
    days_taken_first = (first - pickup).days
    
    #count number of sundays:
    start_day = pickup.weekday()
    quo = days_taken_first % 7
    num_sundays = days_taken_first // 7
        
    if start_day + quo >= 6:
        num_sundays += 1
        
    #date_1 = datetime.strptime(pickup, '%d/%m/%Y')
    #date_2 = datetime.strptime(first, '%d/%m/%Y')
        
        #minus holidays
    for holiday in HOLIDAYS:
        if pickup < holiday < first:
            days_taken_first -= 1
    days_taken_first -= num_sundays
    
    if pd.isnull(row["2nd_deliver_attempt"]):    
        if days_taken_first <= curr_SLA:
            is_late.append(0)
            #print (str(row["orderid"]) + ": " + "0")
        else:
            is_late.append(1)
            #print (str(row["orderid"]) + ": " + "1")
        
        
    else:       
        #second attempt
        second = row["2nd_deliver_attempt"]
        days_taken_sec = (second - first).days
        
        #count number of sundays sec attempt:
        start_day_sec = first.weekday()
        quo_sec = days_taken_sec % 7
        num_sundays_sec = days_taken_sec // 7
        
        if start_day_sec + quo_sec >= 6:
            num_sundays_sec += 1
        
        #date_3 = datetime.strptime(first, '%d/%m/%Y')
        #date_4 = datetime.strptime(second, '%d/%m/%Y')
        
        #minus holidays sec attempt
        for holiday in HOLIDAYS:
            if first < holiday < second:
                days_taken_sec -= 1
                
        days_taken_sec -= num_sundays_sec
        
        
        if (days_taken_first <= curr_SLA and days_taken_sec <= 3):
            is_late.append(0)
            #print (str(row["orderid"]) + ": " + "0")
        else:
            is_late.append(1)
            #print (str(row["orderid"]) + ": " + "1")

new_df = pd.DataFrame({'orderid':order_ids,'is_late':is_late})
new_df.to_csv('output.csv',index=False)